In [1]:
from utils import build_url, load_json, save_json, call_with_retry, save_execution_state, load_execution_state
import os, shutil

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = [420, 430, 440]  # Ver https://developer.riotgames.com/docs/lol#general_queue-ids

with open(API_KEY_FILE) as f:
    api_key = f.read()

execution_cache_dir = 'current_execution'
games_ids_state = f'{execution_cache_dir}/games.pkl'
accounts_state = f'{execution_cache_dir}/accounts.pkl'

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

headers = {
  'X-Riot-Token': api_key
}

accounts = load_json('outputs/accounts.json')
print(accounts[0:5])
accounts_ids = list(map(lambda a: a['account'], accounts))
print(accounts_ids[0:10])

[{'summoner': 'I6FBVa-W-NQRMm1odpG0PkI3sQCbYoQlhE7j6R6aTe2upnI', 'account': 'qkVFtFB-BtPcEq8g9ivcISNtyqxMF-OS38l8H9y7BCyX1g'}, {'summoner': 'jZf8y-91mnbox-V2DHR6TlMwbbXP1-k3BD23SMQk0TgDYAo', 'account': 'lOapMWCGTfdc5Z3HH5WvdAxpABVwZY9CLcD7vYoRq2JrXA'}, {'summoner': 'LkCrNx2rwEp7ouzvSzVZ81ZgW0tLrPqDAXimvHEYwwh-HT5O', 'account': '0NFzdAnsy3Mia_Pup0_0IpLEifeBmzCPKgjb9z3CxyANlRRbQnprDSOT'}, {'summoner': 'CE6Go5eaGb-HffQAR6XpAaoGuFloaOUOO05GEKuzwWIevOH4', 'account': 'c3u0czKdYrEy0BFb_luVDeb_3fp_rVrG7EpqLRYhh15j-pn1i-M_jn4U'}, {'summoner': 'fMAfv3bollmkd8cNPm5ev-tyXE32-mVytWiTrBblrAAYnaY', 'account': '2dw_Tr_dJHAElbKJGjw01P7pC2RuRD1Tx3AFX36bnHAsiw'}]
['qkVFtFB-BtPcEq8g9ivcISNtyqxMF-OS38l8H9y7BCyX1g', 'lOapMWCGTfdc5Z3HH5WvdAxpABVwZY9CLcD7vYoRq2JrXA', '0NFzdAnsy3Mia_Pup0_0IpLEifeBmzCPKgjb9z3CxyANlRRbQnprDSOT', 'c3u0czKdYrEy0BFb_luVDeb_3fp_rVrG7EpqLRYhh15j-pn1i-M_jn4U', '2dw_Tr_dJHAElbKJGjw01P7pC2RuRD1Tx3AFX36bnHAsiw', 'cpDa6rnHFIQP33_P-Thda6DBEFZQQCFYNIbcnClYbXKGrEc', 'gSfZtRohb2uZvXVOCxDugSQJ

In [2]:
endpoint = '/lol/match/v4/matchlists/by-account/{}'
params = {
    'queue': QUEUE_TYPE,
}

games_ids = load_execution_state(games_ids_state, set())
accounts_with_matches = load_execution_state(accounts_state, set())
accounts_ids = set(accounts_ids) - accounts_with_matches

for account_id in accounts_ids:
    full_url = build_url(REGION, BASE_URL, endpoint.format(account_id))
    response = call_with_retry(full_url, headers, params)
    for match in response['matches']:
        games_ids.add(match['gameId'])
    save_execution_state(games_ids, games_ids_state)
    accounts_with_matches.add(account_id)
    save_execution_state(accounts_with_matches, accounts_state)
        
save_json(list(games_ids), 'games_ids.json')

In [3]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)